# 📊 7 Questions : Construire un agent Text-to-SQL avec GPT-4.1

## 🇫🇷 Bienvenue dans ce workshop pratique ! 🇬🇧 Welcome to this hands-on workshop!

Dans ce notebook, nous allons construire **progressivement** un agent intelligent capable de transformer des questions en langage naturel en requêtes SQL, puis d'exécuter ces requêtes et même de créer des visualisations.

### 🎯 Objectifs du workshop
- Comprendre les bases du **Text-to-SQL** avec les LLMs
- Maîtriser la **sortie structurée** avec Pydantic 
- Implémenter un **système de mémoire** conversationnelle
- Créer un **agent autonome** avec function calling

### 🛠️ Prérequis
- Python ≥ 3.10
- Une clé API OpenAI (GPT-4.1 recommandé, GPT-4o compatible)
- Le fichier `catalogue.csv` dans le même dossier

---

## 📖 Section 1 : Introduction & Setup

### Qu'est-ce que le Text-to-SQL ?

Le **Text-to-SQL** est une technique qui permet de convertir des questions en langage naturel en requêtes SQL. Par exemple :

- 🗣️ **Question** : "Combien d'articles rouges avons-nous ?"
- 🔍 **SQL généré** : `SELECT COUNT(*) FROM catalogue WHERE couleur = 'rouge'`

### Pourquoi utiliser des agents ?

Un **agent** peut enchaîner plusieurs outils :
1. 🧠 Générer du SQL à partir d'une question
2. ⚡ Exécuter la requête sur la base de données  
3. 📊 Créer une visualisation si nécessaire
4. 💬 Répondre en langage naturel

C'est exactement ce que nous allons construire !

### 🔧 Installation des dépendances

Commençons par installer les bibliothèques nécessaires :

In [1]:
# Installation des dépendances
!pip install openai>=1.0.0 pandas python-dotenv matplotlib ipython-sql pydantic

zsh:1: 1.0.0 not found


### 🔑 Configuration de la clé API OpenAI

Vous avez deux options pour définir votre clé API :

**Option 1 : Fichier .env (recommandé)**
```bash
# Créez un fichier .env dans le même dossier que ce notebook
OPENAI_API_KEY=sk-your-key-here
```

API ressources : 
- https://platform.openai.com/docs/guides/text?api-mode=chat

In [11]:
# Imports et configuration
import os
import pandas as pd
import sqlite3
import json
from typing import List, Dict, Any
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel
import matplotlib.pyplot as plt
import matplotlib.style as style

# Chargement des variables d'environnement
load_dotenv()

# Configuration du client OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Modèle par défaut (GPT-4.1)
DEFAULT_MODEL = "gpt-4.1-2025-04-14"  # Version stable de GPT-4.1
# Alternative pour les utilisateurs avec GPT-4o uniquement :
# DEFAULT_MODEL = "gpt-4o"

print("✅ Configuration terminée !")
print(f"🤖 Modèle utilisé : {DEFAULT_MODEL}")

# Test de la connexion
try:
    test_response = '...'
    """
    analysez la doc pour faire votre premier appel à l'API OpenAI 
    """
    print("🔗 Connexion à l'API OpenAI réussie !")
except Exception as e:
    print(f"❌ Erreur de connexion : {e}")
    print("💡 Vérifiez votre clé API dans le fichier .env")

✅ Configuration terminée !
🤖 Modèle utilisé : gpt-4.1-2025-04-14
🔗 Connexion à l'API OpenAI réussie !


---

## ⚙️ Question 2 : Première requête – l'LLM renvoie du SQL brut

### 🎯 Objectif
Faire en sorte que GPT-4.1 transforme une question en langage naturel en requête SQL pure, sans explication.

### 📋 Étapes
1. Charger le fichier `catalogue.csv` dans SQLite
2. Analyser le schéma de la base de données  
3. Créer un prompt système optimisé
4. Tester avec une question simple

In [3]:
# Étape 1 : Chargement du catalogue en SQLite
def setup_database():
    """Charge le fichier catalogue.csv dans une base SQLite en mémoire"""
    
    # Lecture du CSV
    try:
        df = pd.read_csv('catalogue.csv')
        print(f"📊 Fichier chargé : {len(df)} lignes, {len(df.columns)} colonnes")
        print(f"🏷️ Colonnes : {list(df.columns)}")
        
        # Affichage des premières lignes
        print("\n🔍 Aperçu des données :")
        print(df.head())
        
    except FileNotFoundError:
        print("❌ Fichier catalogue.csv non trouvé !")
        print("💡 Assurez-vous qu'il soit dans le même dossier que ce notebook")
        return None, None
    
    # Création de la base SQLite en mémoire
    conn = sqlite3.connect(':memory:')
    df.to_sql('catalogue', conn, index=False, if_exists='replace')
    
    print(f"\n✅ Base de données créée avec la table 'catalogue'")
    
    return conn, df

# Exécution
conn, df = setup_database()

📊 Fichier chargé : 590 lignes, 9 colonnes
🏷️ Colonnes : ['external_id', 'reference', 'color', 'tra_value', 'size', 'label', 'ean', 'image', 'price']

🔍 Aperçu des données :
  external_id reference color   tra_value  size           label  \
0  A065H94000     A065H   ALF  variante 7     0  3 BODIES US MC   
1  A065H94010     A065H   ALF  variante 7     1  3 BODIES US MC   
2  A065H94030     A065H   ALF  variante 7     3  3 BODIES US MC   
3  A065H94070     A065H   ALF  variante 7     7  3 BODIES US MC   
4  A065H94080     A065H   ALF  variante 7     8  3 BODIES US MC   

             ean                                              image  price  
0  3666072761781  https://www.petit-bateau.fr/dw/image/v2/BCKL_P...   22.0  
1  3666072761798  https://www.petit-bateau.fr/dw/image/v2/BCKL_P...   22.0  
2  3666072761828  https://www.petit-bateau.fr/dw/image/v2/BCKL_P...   22.0  
3  3666072761811  https://www.petit-bateau.fr/dw/image/v2/BCKL_P...   22.0  
4  3666072761835  https://www.petit-bat

In [ ]:
# Exemple : Exécuter une requête SQL avec le connecteur conn
query = "VOTRE REQUETE SQL ICI"
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()
print("Top 5 couleurs les plus fréquentes :")
for color, count in results:
    print(f"{color}: {count}")

Top 5 couleurs les plus fréquentes :
ZGA: 65
ZG9: 52
BFQ: 45
ZG7: 39
FOZ: 25


## Prenez le temps d'analyser la dataframe df et de comprendre les colonnes disponibles.

In [ ]:
df.head() ...

## Étape 2 : Analyse des données avec OpenAI

In [7]:
schema = """
Définissez ici le schéma de votre base de données pour que le LLM puisse l'utiliser.
précisez le nom des colonnes et leurs types, des exemples de données, etc.

output attendu : string
"""

In [8]:
# Étape 3 : Création du prompt système optimisé
def create_text_to_sql_prompt(schema_info):
    prompt = """Crée un prompt système pour la génération de SQL"""
        
    return prompt

# Création du prompt
text_to_sql_prompt = create_text_to_sql_prompt(schema)
print("📝 Prompt système créé :")
print("=" * 50)
print(text_to_sql_prompt[:300] + "..." if len(text_to_sql_prompt) > 300 else text_to_sql_prompt)

📝 Prompt système créé :
Crée un prompt système pour la génération de SQL


In [9]:
# Étape 4 : Fonction de génération SQL simple
def text_to_sql_basic(question: str) -> str:
    """Convertit une question en requête SQL avec GPT-4.1.
    Utilisez l'api pour requeter OPENAI"""

    sql_query = 'la bonne query'
    return sql_query
    
# Test avec une question simple
test_question = "Donne-moi la requête SQL pour sortir tous les articles rouges"

print(f"🗣️ Question : {test_question}")
print("=" * 60)

sql_result = text_to_sql_basic(test_question)
print(f"🔍 SQL généré :")
print(sql_result)

# Validation rapide de la syntaxe
if sql_result.startswith("SELECT") and sql_result.endswith(";"):
    print("\n✅ La requête semble syntaxiquement correcte !")
else:
    print("\n⚠️ La requête pourrait avoir des problèmes de syntaxe")

🗣️ Question : Donne-moi la requête SQL pour sortir tous les articles rouges
🔍 SQL généré :
la bonne query

⚠️ La requête pourrait avoir des problèmes de syntaxe


### 🧪 Mini-Quiz : Understanding Text-to-SQL

**Question** : Pourquoi demandons-nous à GPT-4.1 de répondre "uniquement par la requête SQL" ?

<details>
<summary>💡 Cliquez pour voir la réponse</summary>

**Réponse** : C'est pour avoir une sortie **déterministe** et **facile à traiter programmatiquement**. Si le modèle ajoute du texte explicatif, il faudrait le parser pour extraire juste le SQL, ce qui est plus complexe et source d'erreurs.

Dans la prochaine section, nous verrons comment **Pydantic** nous permet de structurer encore mieux cette sortie !

</details>

---

## 🗂 Question 3 : Sortie structurée avec Pydantic

### 🎯 Objectif
Utiliser **Pydantic** pour valider et structurer la réponse de GPT-4.1, garantissant un format de sortie cohérent.

### 🌟 Pourquoi Pydantic ?
- ✅ **Validation automatique** des types de données
- 🛡️ **Gestion d'erreur** robuste  
- 🏗️ **Structure prévisible** pour notre code
- 📚 Compatible avec l'API **Structured Outputs** d'OpenAI

### 📖 Ressources
- [OpenAI Structured Outputs Guide](https://platform.openai.com/docs/guides/structured-outputs)
- [Pydantic Documentation](https://docs.pydantic.dev/)

In [ ]:
# Définition du modèle Pydantic pour la sortie structurée
class TextToSQLResponse(BaseModel):
    """Modèle complet pour la réponse text-to-SQL
    definissez les keys necessaires et les bons types"""
    field1: list
    field2 : str
    query : int
    # A compléter avec les champs nécessaires
 

# Affichage du schéma JSON pour comprendre la structure
print("📋 Schéma Pydantic généré :")
print("=" * 40)
print(json.dumps(TextToSQLResponse.model_json_schema(), indent=2))

📋 Schéma Pydantic généré :
{
  "description": "Mod\u00e8le complet pour la r\u00e9ponse text-to-SQL\ndefinissez les keys necessaires et les bons types",
  "properties": {
    "field1": {
      "items": {},
      "title": "Field1",
      "type": "array"
    },
    "field2": {
      "title": "Field2",
      "type": "string"
    },
    "query": {
      "title": "Query",
      "type": "integer"
    }
  },
  "required": [
    "field1",
    "field2",
    "query"
  ],
  "title": "TextToSQLResponse",
  "type": "object"
}


In [23]:
#rappel
text_to_sql_prompt[:100]

'Crée un prompt système pour la génération de SQL'

Pour cette question, utiliser le endpoint decrit dans cette doc :
- https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat

In [ ]:
def text_to_sql_structured(question: str) -> TextToSQLResponse:
    response = client.responses.parse(
       # a completer
    )
    return response.output_parsed

# Test de la fonction structurée
test_question = "Combien d'articles de couleur bleue avons-nous ?"

print(f"🗣️ Question : {test_question}")
print("=" * 60)

structured_result = text_to_sql_structured(test_question)

print(f"🔍 Requête SQL : {structured_result.query}")

# Validation du type
print(f"\n✅ Type de retour : {type(structured_result)}")
print(f"🏗️ Validation Pydantic : {'Réussie' if structured_result.query else 'Échouée'}")


In [ ]:
structured_result.query


In [ ]:
# Comparaison entre l'approche basique et structurée
print("🔬 COMPARAISON DES DEUX APPROCHES")
print("=" * 60)

test_questions = [
    "Quels sont les 5 articles les plus chers ?",
    "Donne-moi la moyenne des prix par catégorie",
    "Combien d'articles contiennent le mot 'premium' ?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 Test {i}: {question}")
    print("-" * 40)
    
    # Approche basique
    basic_result = text_to_sql_basic(question)
    print(f"🟡 Basique: {basic_result}")
    
    # Approche structurée  
    struct_result = text_to_sql_structured(question)
    print(f"🟢 Structurée: {struct_result.query}")


In [ ]:
structured_result.model_dump_json()

---

## 🧠 Question 4 : Adding Memory (Conversation History)

### 🎯 Objectif  
Maintenir un **historique de conversation** pour que l'agent puisse se souvenir des interactions précédentes et fournir des réponses plus contextuelles.

Exemple : 
- Quels sont les articles rouges?
- Combien valent plus de 20 Euros?

### 🔄 Pourquoi la mémoire est importante ?
- **Continuité** : "Maintenant trie par prix" fait référence à la requête précédente
- **Contexte** : L'agent comprend les questions de suivi
- **Expérience utilisateur** : Conversation plus naturelle

### 💡 Approche
Maintenir une liste de messages qui grandit au fil de la conversation.

In [ ]:
# Classe pour gérer la mémoire conversationnelle
class ConversationalSQLAgent:
    """Agent Text-to-SQL avec mémoire conversationnelle"""
    
    def __init__(self, system_prompt: str):
        # Ajoute une consigne pour la prise en compte du contexte conversationnel
        #TODO
        historical_prompt = (
            system_prompt
            + "AJUSTER LE SYSTEME PROMPT POUR PRENDRE EN COMPTE L'HISTORIQUE DE LA CONVERSATION"
        )
        self.system_prompt = historical_prompt
        self.conversation_history = [
            {"role": "system", "content": historical_prompt}
            # tous les autres messages seront ajoutés ici sous la forme de dictionnaires en precisant le role
            # message de l'utilisatuer : user, reponse de l'AI : assistant
        ]
        self.query_count = 0
    
    def add_user_message(self, message: str):
        """Ajoute un message utilisateur à l'historique - ATTENTION A BIEN PRECISER LE ROLE USER DANS LE DICT"""
        self.conversation_history.append({"role": "user", "content": message})
    
    def add_assistant_message(self, message: str):
        """Ajoute une réponse de l'assistant à l'historique"""
        #TODO
    
    def generate_sql(self, question: str) -> TextToSQLResponse:
        #TODO
        """Génère du SQL en tenant compte de l'historique"""
        
        # Ajouter la question actuelle de l'utilisateur à l'historique
        self.add_user_message(question)
        
        try:
            # Utilise la fonction structurée pour obtenir la réponse.
            # L'historique est a jour avec la toute derniere question de l;utilisateur
            structured_result = client.responses.parse(...         #TODO
            
            # sauve la reponse du LLM dans l'historique en mode assistant. Ne pas oublier de formatter le BAseModel en JSON 
            self.add_assistant_message(structured_result.model_dump_json())
            self.query_count += 1
            return structured_result
            
        except Exception as e:
            error_response = TextToSQLResponse(
                query="SELECT 1; -- Erreur"
            )
            self.add_assistant_message(error_response.model_dump_json())
            return error_response
    
    def get_history_summary(self) -> str:
        """Résumé de l'historique de conversation"""
        return f"Conversation: {len(self.conversation_history)} messages, {self.query_count} requêtes générées"
    
    def clear_history(self):
        """Remet à zéro l'historique (garde le prompt système)"""
        self.conversation_history = [self.conversation_history[0]]  # Garde seulement le système
        self.query_count = 0

# Initialisation de l'agent conversationnel
conversational_agent = ConversationalSQLAgent(text_to_sql_prompt)
print("🤖 Agent conversationnel initialisé !")
print(f"📊 État initial : {conversational_agent.get_history_summary()}")


🤖 Agent conversationnel initialisé !
📊 État initial : Conversation: 1 messages, 0 requêtes générées


In [ ]:
# Test de la mémoire conversationnelle avec reset
print("🎭 DÉMONSTRATION DE LA MÉMOIRE CONVERSATIONNELLE (avec reset)")
print("=" * 60)

# Séquence de questions qui s'appuie sur le contexte
conversation_sequence = [
    "Montre-moi tous les articles plus chers que 20 euros",
    "Maintenant trie-les par prix décroissant", 
    "Limite à 3 résultats",
    "Ajoute aussi les articles bleus à cette sélection"
]

for i, question in enumerate(conversation_sequence, 1):
    print(f"\n💬 Question {i}: {question}")
    print("-" * 30)
    
    result = conversational_agent.generate_sql(question)
    
    print(f"🔍 SQL: {result.query}")
    print(f"📝 Historique: {conversational_agent.get_history_summary()}")
    

    # teste manuellement l'exécution de la requête SQL - pour verifier la validité
    try:
        cursor = conn.cursor()
        cursor.execute(result.query)
        rows = cursor.fetchall()
        print(f"✅ Exécution réussie: {len(rows)} résultats")
    except Exception as e:
        print(f"❌ Erreur d'exécution: {e}")


In [ ]:
print("\n🔄 Maintenant, on va faire quelque chose de nouveau : on réinitialise la mémoire de l'agent !")
conversational_agent.clear_history()
print(f"🧹 Historique après reset: {conversational_agent.get_history_summary()}")

# Nouvelle séquence de questions indépendantes
new_questions = [
    "Combien d'articles coûtent moins de 10 euros ?",
    "Quels sont les 2 articles les moins chers ?"
]

for i, question in enumerate(new_questions, 1):
    print(f"\n💬 Nouvelle question {i}: {question}")
    print("-" * 30)
    
    result = conversational_agent.generate_sql(question)
    
    print(f"🔍 SQL: {result.query}")
    print(f"📝 Historique: {conversational_agent.get_history_summary()}")
    
    try:
        cursor = conn.cursor()
        cursor.execute(result.query)
        rows = cursor.fetchall()
        print(f"✅ Exécution réussie: {len(rows)} résultats")
    except Exception as e:
        print(f"❌ Erreur d'exécution: {e}")

print(f"\n🎯 BILAN FINAL")
print(f"📊 {conversational_agent.get_history_summary()}")
print("✨ L'agent a bien réinitialisé et géré deux contextes séparés !")

---

## 🔧 Question 5 : Function / Tool Calling

### 🎯 Objectif
Implémenter le **Function Calling** (Tool Calling) d'OpenAI pour que l'agent puisse :
1. 🧠 **Générer** des requêtes SQL (`make_sql`)
2. ⚡ **Exécuter** les requêtes sur la base (`run_query`)

### 🛠️ Concept de Function Calling
Le modèle GPT-4.1 peut décider quand et comment utiliser des outils externes. Il reçoit une description des outils disponibles et choisit lequel utiliser en fonction du contexte.

### 📚 Flux de travail
1. **Question utilisateur** : "Combien d'articles rouges ?"
2. **GPT-4.1 décide** : J'ai besoin de `make_sql` 
3. **Exécution** : L'outil génère `SELECT COUNT(*) FROM catalogue WHERE couleur = 'rouge';`

--- 
1. **Question utilisateur** : "execute la query SELECT * FROM TABLE"
2. **GPT-4.1 décide** : J'ai besoin de runner `run_query_tool` 
3. **Exécution** : L'outil execute la function avec l'argument `SELECT * FROM TABLE;`




Ressources : 
- https://platform.openai.com/docs/guides/function-calling?api-mode=chat

In [ ]:
# Définition des outils (tools) disponibles pour l'agent

def make_sql_tool(question: str) -> str:
    #TODO

def run_query_tool(sql_query: str) -> str:
    """
    Outil : Exécute une requête SQL sur la base de données
    """
    
    #TODO
    df_result = pd.DataFrame(rows, columns=columns)
    
    if len(df_result) == 0:
        return "Aucun résultat trouvé."
    elif len(df_result) <= 10:
        return f"Résultats ({len(df_result)} lignes):\\n{df_result.to_string(index=False)}"
    else:
        return f"Résultats ({len(df_result)} lignes, affichage des 10 premières):\\n{df_result.head(10).to_string(index=False)}"


# Définition des outils au format OpenAI Function Calling
tools_definition = [
   #TODO
]

print("🔧 Outils définis avec succès !")
print("📋 Outils disponibles :")
for tool in tools_definition:
    func = tool["function"]
    print(f"  • {func['name']}: {func['description']}")

In [ ]:
# Agent avec Function Calling
class ToolCallingAgent:
    """Agent qui utilise les outils via Function Calling """
    
    def __init__(self):
        self.tools = tools_definition
        self.conversation_history = []
        self.available_functions = {
            "make_sql": make_sql_tool,
            "run_query": run_query_tool
        }
    
    def chat(self, user_message: str) -> str:
        """Interface principale pour discuter avec l'agent """
        
        # Ajouter le message utilisateur
        self.conversation_history.append({"role": "user", "content": user_message})
        
        # Prompt système pour l'agent
        #TODO
        system_prompt = f"""modifier le systemen prompt pour prendre en compte les outils disponibles"""
        
        # Préparer les messages avec le système
        messages = [{"role": "system", "content": system_prompt}] + self.conversation_history
        
        try:
            # Appel initial avec les outils
            response = client.chat.completions.create(
                #TODO
            )
            
            response_message = response.choices[0].message
            
            # Vérifier si le modèle veut utiliser des outils
            if response_message.tool_calls:
                # Ajouter la réponse du modèle à l'historique
                self.conversation_history.append({
                    "role": "assistant", 
                    "content": response_message.content,
                    "tool_calls": response_message.tool_calls
                })
                
                # Exécuter chaque outil demandé
                for tool_call in response_message.tool_calls:
                    function_name = #get the function name from the tool call
                    function_args = #get the json args from the tool call
                    
                    print(f"🔧 Utilisation de l'outil : {function_name}")
                    print(f"📥 Arguments : {function_args}")
                    
                    # Exécuter la fonction
                    if function_name in self.available_functions:
                        function_result = self.available_functions[function_name](**function_args)
                        print(f"📤 Résultat : {function_result[:100]}{'...' if len(str(function_result)) > 100 else ''}")
                        
                        # Ajouter le résultat à l'historique
                        self.conversation_history.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool", 
                            "name": function_name,
                            "content": str(function_result)
                        })
                
                # Nouvel appel pour obtenir la réponse finale
                final_response = client.chat.completions.create(
                    #prendre en compte l'historique mis à jour avec la derniere reponse de l'outil
                )
                
                final_message = final_response.choices[0].message.content
                self.conversation_history.append({"role": "assistant", "content": final_message})
                
                return final_message
            
            else:
                # Pas d'outils utilisés, réponse directe
                self.conversation_history.append({"role": "assistant", "content": response_message.content})
                return response_message.content
                
        except Exception as e:
            error_msg = f"Erreur lors du traitement : {e}"
            self.conversation_history.append({"role": "assistant", "content": error_msg})
            return error_msg



In [ ]:
# Création de l'agent avec tools
tool_agent = ToolCallingAgent()
print("🤖 Agent avec Function Calling initialisé !")

# Test du Function Calling avec un dialogue complet
print("🎪 DÉMONSTRATION DU FUNCTION CALLING")
print("=" * 60)

test_questions = [
    "donne moi la query pour sortir tous les articles plus chers que 15 euros",
    "execute la query SELECT * FROM catalogue ORDER BY price ASC LIMIT 2;",
    "Combien d'articles rouges avons-nous ?",
    "Parmi ceux la, donne moi les 3 articles les plus chers",
]

for i, question in enumerate(test_questions, 1):
    print(f"\n🗣️ Question {i}: {question}")
    print("=" * 40)
    
    response = tool_agent.chat(question)
    
    print(f"\n🤖 Réponse finale:")
    print(response)
    print("\n" + "-" * 40)

---

## 🤖 Question 6 : Building a Simple Agent Loop

### 🎯 Objectif
Créer un **agent autonome** qui tourne en boucle et peut traiter plusieurs demandes de l'utilisateur de manière interactive.

### 🔄 Concept d'Agent Loop
Un agent en boucle peut :
- Attendre des commandes utilisateur
- Traiter les demandes de manière autonome
- Maintenir le contexte entre les interactions
- Gérer les erreurs gracieusement
- Permettre à l'utilisateur de sortir proprement

Exemple : quels sont les articles a moins de 30 Euros ?
- generer une query
- executer la query
- repondre a l'utilisateur

On est plus limite a un seul call de fonction

In [ ]:
# Agent avec Function Calling amélioré : boucle jusqu'à satisfaction de la requête utilisateur
class ToolCallingAgent:
    """Agent qui utilise les outils via Function Calling, avec boucle jusqu'à résultat final"""

    def __init__(self):
        self.tools = tools_definition
        self.conversation_history = []
        self.available_functions = {
            "make_sql": make_sql_tool,
            "run_query": run_query_tool
        }

    def reset_history(self):
        """Réinitialise l'historique de conversation"""
        self.conversation_history = []
        print("🧹 Historique réinitialisé !")
    
    def chat(self, user_message: str) -> str:
        """Interface principale pour discuter avec l'agent (boucle jusqu'à réponse finale)"""

        self.conversation_history.append({"role": "user", "content": user_message})

        system_prompt = " Definir le systeme prompt pour l'agent manager qui va appeler les differents outils " \
        "successivement jusqu'à obtenir une réponse satisfaisante. " 

        messages = [{"role": "system", "content": system_prompt}] + self.conversation_history

        try:
            while True:
                response = client.chat.completions.create(
                   #TODO: utiliser le modèle et les messages avec les outils disponibles
                )
                response_message = response.choices[0].message

                # Si le modèle demande un ou plusieurs outils
                if getattr(response_message, "tool_calls", None):
                    self.conversation_history.append({
                        "role": "assistant",
                        "content": response_message.content,
                        "tool_calls": response_message.tool_calls
                    })
                    for tool_call in response_message.tool_calls:
                        function_name = # TODO get the function name from the tool call
                        function_args = # TODO get the json args from the tool call
                        print(f"🔧 Utilisation de l'outil : {function_name}")
                        print(f"📥 Arguments : {function_args}")

                        if function_name in self.available_functions:
                            function_result = self.available_functions[function_name](**function_args)
                            print(f"📤 Résultat : {str(function_result)[:100]}{'...' if len(str(function_result)) > 100 else ''}")
                            self.conversation_history.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": str(function_result)
                            })
                    # Rafraîchir les messages pour la prochaine boucle
                    messages = [{"role": "system", "content": system_prompt}] + self.conversation_history
                    continue  # Boucle tant qu'il y a des tool_calls

                # Si pas d'outils demandés, réponse finale
                self.conversation_history.append({"role": "assistant", "content": response_message.content})
                return response_message.content

        except Exception as e:
            error_msg = f"Erreur lors du traitement : {e}"
            self.conversation_history.append({"role": "assistant", "content": error_msg})
            return error_msg


In [ ]:
interactive_agent = ToolCallingAgent()

# Test automatique de l'agent (simulation d'une session)
print("🎭 SIMULATION D'UNE SESSION INTERACTIVE")
print("=" * 60)

# Simulation de commandes utilisateur
simulated_session = [
    "Combien d'articles avons-nous au total ?",
    "Montre-moi les articles contenant le mot 'bodies'",
    "Parmi ces articles, quels sont les 3 plus chers ?",
]

print("🤖 Démarrage de la simulation...")
print("📝 Commandes à tester :", simulated_session)
print("\\n" + "=" * 40)

for i,msg in enumerate(simulated_session):
    print('')
    print('')
    print(f"\\n🎬 Simulation {i}: {msg}")
    print("-" * 30)
    
    # Traitement de la commande
    agent_answer = interactive_agent.chat(msg)
    print(f"🤖 Réponse de l'agent : {agent_answer}")
    print("-" * 30)
    
    if not agent_answer:
        print("⚠️ L'agent a demandé l'arrêt")
        break

print("\\n🎬 Fin de la simulation")
print("💡 Pour une session réelle, exécutez : interactive_agent.run_interactive_loop()")

In [ ]:
interactive_agent = ToolCallingAgent()

# Test automatique de l'agent (simulation d'une session)
print("🎭 SIMULATION D'UNE SESSION INTERACTIVE")
print("=" * 60)

# Simulation de commandes utilisateur
simulated_session = [
    "regarde successivement le nombre d'articles par couleur, puis la couleur de l'article le plus cher, et enfin les articles de cette couleur. ",
]

print("🤖 Démarrage de la simulation...")
print("📝 Commandes à tester :", simulated_session)
print("\\n" + "=" * 40)

for i,msg in enumerate(simulated_session):
    print(f"\\n🎬 Simulation {i}: {msg}")
    print("-" * 30)
    
    # Traitement de la commande
    should_continue = interactive_agent.chat(msg)
    
    if not should_continue:
        print("⚠️ L'agent a demandé l'arrêt")
        break

print("\\n🎬 Fin de la simulation")
print("💡 Pour une session réelle, exécutez : interactive_agent.run_interactive_loop()")




## 📊 Question 7 : Simplify it using the Agents SDK


Benefices: 
 - Gestion automatique du contexte
 - Gestion automatique de la boucle de l'agent, on definit uniquement les tools 

Resources : 
- https://openai.github.io/openai-agents-python/
- https://github.com/openai/openai-agents-python/blob/main/examples/basic/tools.py

In [ ]:
pip install openai-agents

In [ ]:
from agents import Agent, Runner, function_tool
import pandas as pd  # Ensure this is imported
import asyncio

# Assume conn and text_to_sql_structured are globally available
# e.g. conn = sqlite3.connect('your_db.sqlite') or similar

@function_tool
def make_sql(question: str) -> str:
    # TODO

@function_tool
def run_query(sql_query: str) -> str:
   #TODO

# Define the agent
sql_agent = Agent(
    name="SQL Assistant",
    instructions=(
        "your system prompt manager here"
    ),
    tools=[make_sql, run_query],
)

result = await Runner.run(sql_agent, "how many items below 10 euros do we have? ")  # type: ignore[top-level-await]  # noqa: F704
print(result.final_output)


# Showing streamed items

In [ ]:
import asyncio
import pandas as pd
from agents import Agent, Runner, function_tool, ItemHelpers

# Assume conn and text_to_sql_structured are defined globally
# e.g. conn = sqlite3.connect('your_db.sqlite')


async def main(input_query):
    sql_agent = Agent(
        name="SQL Assistant",
        instructions=(
        "your system prompt manager here"
    ),
        tools=[make_sql, run_query],
    )

    result = Runner.run_streamed(sql_agent,input_query)
    
    print("=== Run starting ===")
    async for event in result.stream_events():
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
        elif event.type == "run_item_stream_event":
            item = event.item
            if item.type == "tool_call_item":
                print(f"-- Tool called: {item.raw_item.name}")
                print(f"   With input: {item.raw_item.arguments}")
            elif item.type == "tool_call_output_item":
                print(f"-- Tool output: {item.output}")
            elif item.type == "message_output_item":
                print(f"-- Message output:\n{ItemHelpers.text_message_output(item)}")
            print('')
    print("=== Run complete ===")

if __name__ == "__main__":
    asyncio.run(main("how many items below 10 euros do we have? "))  # type: ignore[top-level-await]  # noqa: F704
    print('==========')
    print('==========')

    asyncio.run(main("how many of them are red? "))  # type: ignore[top-level-await]  # noqa: F704

=== Run starting ===
Agent updated: SQL Assistant
-- Tool called: make_sql
   With input: {"question":"how many items below 10 euros do we have?"}

-- Tool output: SELECT COUNT(*) FROM catalogue WHERE price < 10;

-- Tool called: run_query
   With input: {"sql_query":"SELECT COUNT(*) FROM catalogue WHERE price < 10;"}

-- Tool output: Résultats (1 lignes):
 COUNT(*)
        3

-- Message output:
We have 3 items priced below 10 euros.

=== Run complete ===
=== Run starting ===
Agent updated: SQL Assistant
-- Tool called: make_sql
   With input: {"question":"How many products are red?"}

-- Tool output: SELECT COUNT(*) FROM catalogue WHERE color LIKE '%red%';

-- Tool called: run_query
   With input: {"sql_query":"SELECT COUNT(*) FROM catalogue WHERE color LIKE '%red%';"}

-- Tool output: Résultats (1 lignes):
 COUNT(*)
        0

-- Message output:
There are no red products in the catalog.

=== Run complete ===


# Maintainting conversation context between two conversations

In [78]:
# Alternative: Create a conversation manager class
class ConversationManager:
    def __init__(self, agent):
        self.agent = agent
        self.conversation_history = []
    
    async def ask(self, question):
        # Add the new user message to conversation history
        if self.conversation_history:
            new_input = self.conversation_history + [{"role": "user", "content": question}]
        else:
            new_input = question
        
        print(f"=== Processing: {question} ===")
        result = Runner.run_streamed(self.agent, new_input)
        
        async for event in result.stream_events():
            if event.type == "raw_response_event":
                continue
            elif event.type == "agent_updated_stream_event":
                print(f"Agent updated: {event.new_agent.name}")
            elif event.type == "run_item_stream_event":
                item = event.item
                if item.type == "tool_call_item":
                    print(f"-- Tool called: {item.raw_item.name}")
                    print(f"   With input: {item.raw_item.arguments}")
                elif item.type == "tool_call_output_item":
                    print(f"-- Tool output: {item.output}")
                elif item.type == "message_output_item":
                    print(f"-- Message output:\n{ItemHelpers.text_message_output(item)}")
                print('')
        
        # Update conversation history for next turn
        self.conversation_history = result.to_input_list()
        print("=== Processing Complete ===")
        return result

async def main_with_manager():
    sql_agent = Agent(
        name="SQL Assistant",
        instructions=(
            "You are an expert SQL assistant helping to analyze a product catalog database.\n"
            "Use the 'make_sql' tool to generate SQL queries from questions.\n"
            "Use the 'run_query' tool to execute SQL queries.\n"
            "Iterate as needed until you can provide a final answer to the user."
        ),
        tools=[make_sql, run_query],
    )
    
    conversation = ConversationManager(sql_agent)
    
    # Now these calls will maintain context
    await conversation.ask("how many items below 10 euros do we have?")
    await conversation.ask("how many of them are red?")
    await conversation.ask("what about blue ones?")

if __name__ == "__main__":

    
    # Option 2: Using conversation manager
    asyncio.run(main_with_manager())

=== Processing: how many items below 10 euros do we have? ===
Agent updated: SQL Assistant
-- Tool called: make_sql
   With input: {"question":"How many items are priced below 10 euros in the product catalog?"}

-- Tool output: SELECT COUNT(*) FROM catalogue WHERE price < 10;

-- Tool called: run_query
   With input: {"sql_query":"SELECT COUNT(*) FROM catalogue WHERE price < 10;"}

-- Tool output: Résultats (1 lignes):
 COUNT(*)
        3

-- Message output:
We have 3 items priced below 10 euros in the catalog.

=== Processing Complete ===
=== Processing: how many of them are red? ===
Agent updated: SQL Assistant
-- Tool called: make_sql
   With input: {"question":"How many red items are priced below 10 euros in the product catalog?"}

-- Tool called: make_sql
   With input: {"question":"How many red items are in the product catalog?"}

-- Tool output: SELECT COUNT(*) FROM catalogue WHERE color LIKE '%red%' AND price < 10;

-- Tool output: SELECT COUNT(*) FROM catalogue WHERE color LIK

---

## 🎊 Félicitations ! Workshop terminé avec succès !

### 🏆 Ce que vous avez accompli

1. **📖 Compréhension des bases** : Text-to-SQL, agents, et GPT-4.1
2. **⚙️ Génération SQL** : Conversion de langage naturel en requêtes SQL
3. **🗂 Sortie structurée** : Validation robuste avec Pydantic
4. **🧠 Mémoire conversationnelle** : Contexte persistant entre les interactions
5. **🔧 Function Calling** : Outils autonomes pour SQL et exécution  
6. **🤖 Agent en boucle** : Interface interactive complète
7. **📊 Openai Agents SDK ** : Comments Simplifier le code

### 🚀 Comment utiliser votre agent

```python
# Agent simple (questions individuelles)
response = advanced_agent.chat("Montre-moi un graphique des ventes par région")

# Agent interactif (session complète)
interactive_agent = InteractiveAgent()
interactive_agent.run_interactive_loop()
```

### 💡 Extensions possibles

- **🌐 API REST** : Exposer l'agent via FastAPI
- **🔐 Authentification** : Gestion des utilisateurs
- **💾 Persistance** : Sauvegarde de l'historique
- **📈 Analytics** : Métriques d'usage de l'agent
- **🎨 Interface Web** : Frontend avec Streamlit ou React

### 📚 Ressources pour aller plus loin

- **OpenAI Function Calling** : [Documentation officielle](https://platform.openai.com/docs/guides/function-calling)
- **Pydantic V2** : [Guide complet](https://docs.pydantic.dev/latest/)
- **Text-to-SQL avancé** : Recherchez "few-shot learning" et "RAG pour SQL"

---

**🎯 Merci d'avoir suivi ce workshop ! Vous maîtrisez maintenant la création d'agents Text-to-SQL avec GPT-4.1.**

*👨‍💻 Happy coding! 🚀*